In [9]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

import os
import torch

import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import random_split
from torch_geometric.data import DataLoader

from spatial.models.monet_ae import MonetDense
from spatial.predict_xenium import test

import hydra
from hydra.experimental import compose, initialize

os.environ["MKL_NUM_THREADS"]="1"
os.environ["NUMEXPR_NUM_THREADS"]="1"
os.environ["OMP_NUM_THREADS"]="1"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [10]:
# Initialize an empty dictionary to store the results
test_loss_rad_dict = {}

In [ ]:
# ckpt_dir = "/nfs/turbo/lsa-regier/scratch/roko/output/lightning_logs/checkpoints/MonetDenseXenium"

# # Iterate through all files in the directory
# for filename in os.listdir(ckpt_dir):
#     # Check if the file is a checkpoint file
#     if filename.endswith('.ckpt'):
#         model_name, hidden_layers, num_kernels, radius, run_name, optim, num_splits = filename.split("__")
#         num_splits = num_splits.removesuffix(".ckpt")
#         num_splits = num_splits.removeprefix("NUM_SPLITS=")
#         fold = run_name[-1]
#         if '[512, 512' not in hidden_layers or "POLAR_AND_DEGREE" not in run_name:
#             continue
#         with initialize(config_path="../config"):
#             # try:
#                 try:
#                     cfg_from_terminal = compose(config_name="configXenium")
#                     OmegaConf.update(cfg_from_terminal, "paths.data", "../data")
#                     OmegaConf.update(cfg_from_terminal, "paths.root", "/nfs/turbo/lsa-regier/scratch/roko")
#                     OmegaConf.update(cfg_from_terminal, "model.kwargs.kernel_size", int(num_kernels))
#                     OmegaConf.update(cfg_from_terminal, "model.kwargs.hidden_dimensions", eval(hidden_layers))
#                     OmegaConf.update(cfg_from_terminal, "training.logger_name", run_name)
#                     OmegaConf.update(cfg_from_terminal, "training.trainer.strategy", "auto")
#                     OmegaConf.update(cfg_from_terminal, "radius", int(radius))
#                     OmegaConf.update(cfg_from_terminal, "gpus", [0])
#                     OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", int(num_splits))
#                     print(cfg_from_terminal.training.filepath)
#                     # Check if the key already exists in the dictionary
#                     if int(radius) in test_loss_rad_dict and int(num_kernels) in test_loss_rad_dict[int(radius)] and int(fold) in test_loss_rad_dict[int(radius)][int(num_kernels)]:
#                         print(f"Key ({int(radius)}, {int(num_kernels)}, {int(fold)}) already exists. Skipping test.")
#                         continue
#                     output = test(cfg_from_terminal)
#                     trainer, l1_losses, inputs_BASE, gene_expressions_BASE, celltypes, test_results_BASE = output
#                     # Store the results over (radius, num_kernels) in a nested dict
#                     if int(radius) not in test_loss_rad_dict:
#                         test_loss_rad_dict[int(radius)] = {}
#                     if int(num_kernels) not in test_loss_rad_dict[int(radius)]:
#                         test_loss_rad_dict[int(radius)][int(num_kernels)] = {}
#                     test_loss_rad_dict[int(radius)][int(num_kernels)][int(fold)] = test_results_BASE[0]['test_loss']
#                 except Exception as e:
#                     if "CUDA out of memory" in str(e):
#                         OmegaConf.update(cfg_from_terminal, "training.filepath", "${model.name}__${model.kwargs.kernel_size}__${radius}__${training.logger_name}__${optimizer.name}__NUM_SPLITS=" + str(num_splits))
#                         OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", 5)
#                         print("Increasing splits to 5 due to OOM error.")
#                         output = test(cfg_from_terminal)
#                         trainer, l1_losses, inputs_BASE, gene_expressions_BASE, celltypes, test_results_BASE = output
#                         # Store the results over (radius, num_kernels) in a nested dict
#                         if int(radius) not in test_loss_rad_dict:
#                             test_loss_rad_dict[int(radius)] = {}
#                         if int(num_kernels) not in test_loss_rad_dict[int(radius)]:
#                             test_loss_rad_dict[int(radius)][int(num_kernels)] = {}
#                         test_loss_rad_dict[int(radius)][int(num_kernels)][int(fold)] = test_results_BASE[0]['test_loss']
#                     else:
#                         raise e


In [ ]:
# Sort the test_loss_rad_dict by radius and then by fold
sorted_test_loss_rad_dict = {k: v for k, v in sorted(test_loss_rad_dict.items(), key=lambda item: item[0])}
for radius in sorted_test_loss_rad_dict:
    sorted_test_loss_rad_dict[radius] = {k: v for k, v in sorted(sorted_test_loss_rad_dict[radius][10].items(), key=lambda item: item[0])}
sorted_test_loss_rad_dict

In [ ]:
# # Calculate the mean and standard deviation of test losses over folds for each radius, sorted by radius
# mean_std_over_folds = {}
# # First, sort the radius keys from lowest to highest
# sorted_radius_keys = sorted(test_loss_rad_dict.keys())
# for radius in sorted_radius_keys:
#     kernel_losses = test_loss_rad_dict[radius]
#     for kernel, fold_losses in kernel_losses.items():
#         mean_loss = np.mean(list(fold_losses.values()))
#         std_loss = np.std(list(fold_losses.values()))
#         if radius not in mean_std_over_folds:
#             mean_std_over_folds[radius] = {}
#         mean_std_over_folds[radius] = {"mean": mean_loss, "std": std_loss}
# mean_std_over_folds

In [2]:
NON_RESPONSE_FILE = "../spatial/non_response_blank_removed_xenium.txt"
with open(NON_RESPONSE_FILE, "r", encoding="utf-8") as f:
    non_response_genes = f.read().split(',')

non_response_genes = [int(x) for x in non_response_genes]

In [ ]:
non_response_genes_set = set(non_response_genes)
response_genes_indexes = [i for i in range(len(inputs_BASE[0])) if i not in non_response_genes_set]
((inputs_BASE[:, response_genes_indexes] - gene_expressions_BASE) ** 2).mean()

In [ ]:
inputs_BASE[:, response_genes_indexes].mean(axis=1), gene_expressions_BASE.mean(axis=1)

In [ ]:
import matplotlib.pyplot as plt

# Initialize a dictionary to store the losses for each kernel value
kernel_losses = {}

# Iterate through the test_loss_rad_dict to populate kernel_losses
for radius, radius_losses in sorted(test_loss_rad_dict.items()):
    for kernel, loss in radius_losses.items():
        if kernel not in kernel_losses:
            kernel_losses[kernel] = []
        kernel_losses[kernel].append((radius, loss))

# Plot the losses for each kernel value
for kernel, losses in kernel_losses.items():
    radii, losses = zip(*losses)  # Unpack the list of tuples into two separate tuples
    plt.plot(radii, losses, label=f"Kernel: {kernel}", marker='o')

plt.xlabel("Radius")
plt.ylabel("Test Loss")
plt.title("Test Loss for Different Kernel Values and Radius")
plt.xticks(range(0, 41, 5))  # Adjust x-axis tick marks to be less frequent
plt.legend()
plt.tight_layout()

In [ ]:
losses

In [ ]:
test_loss_rad_dict = {}

ckpt_dir = "/nfs/turbo/lsa-regier/scratch/roko/output/lightning_logs/checkpoints/MonetDenseXenium"

# Iterate through all files in the directory
for filename in os.listdir(ckpt_dir):
    # Check if the file is a checkpoint file
    if filename.endswith('.ckpt'):
        model_name, hidden_layers, num_kernels, radius, run_name, optim, num_splits = filename.split("__")
        num_splits = num_splits.removesuffix(".ckpt")
        num_splits = num_splits.removeprefix("NUM_SPLITS=")
        if "[512, 512, 512]" not in hidden_layers or "POLAR_AND_DEGREE" != run_name or int(radius) != 0 or int(num_kernels) != 10:
            continue
        with initialize(config_path="../config"):
            # try:
                try:
                    cfg_from_terminal = compose(config_name="configXenium")
                    OmegaConf.update(cfg_from_terminal, "paths.data", "../data")
                    OmegaConf.update(cfg_from_terminal, "paths.root", "/nfs/turbo/lsa-regier/scratch/roko")
                    OmegaConf.update(cfg_from_terminal, "model.kwargs.kernel_size", int(num_kernels))
                    OmegaConf.update(cfg_from_terminal, "model.kwargs.hidden_dimensions", eval(hidden_layers))
                    OmegaConf.update(cfg_from_terminal, "training.logger_name", run_name)
                    OmegaConf.update(cfg_from_terminal, "training.trainer.strategy", "auto")
                    OmegaConf.update(cfg_from_terminal, "radius", int(radius))
                    OmegaConf.update(cfg_from_terminal, "gpus", [1])
                    OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", int(num_splits))
                    print(cfg_from_terminal.training.filepath)
                    # Check if the key already exists in the dictionary
                    if int(radius) in test_loss_rad_dict and int(num_kernels) in test_loss_rad_dict[int(radius)]:
                        print(f"Key ({int(radius)}, {int(num_kernels)}) already exists. Skipping test.")
                        continue
                    output = test(cfg_from_terminal)
                    trainer, l1_losses, inputs_BASE, gene_expressions_BASE, celltypes, test_results_BASE = output
                    # Store the results over (radius, num_kernels) in a nested dict
                    if int(radius) not in test_loss_rad_dict:
                        test_loss_rad_dict[int(radius)] = {}
                    test_loss_rad_dict[int(radius)][int(num_kernels)] = test_results_BASE[0]['test_loss']
                except Exception as e:
                    if "CUDA out of memory" in str(e):
                        OmegaConf.update(cfg_from_terminal, "training.filepath", "${model.name}__${model.kwargs.kernel_size}__${radius}__${training.logger_name}__${optimizer.name}__NUM_SPLITS=" + str(num_splits))
                        OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", 5)
                        print("Increasing splits to 5 due to OOM error.")
                        output = test(cfg_from_terminal)
                        trainer, l1_losses, inputs_BASE, gene_expressions_BASE, celltypes, test_results_BASE = output
                        # Store the results over (radius, num_kernels) in a nested dict
                        if int(radius) not in test_loss_rad_dict:
                            test_loss_rad_dict[int(radius)] = {}
                        test_loss_rad_dict[int(radius)][int(num_kernels)] = test_results_BASE[0]['test_loss']
                    else:
                        raise e


MonetDense__[512, 512, 512]__10__10__POLAR_AND_DEGREE__Adam__NUM_SPLITS=3
There are 64 graph splits for the Xenium Dataset.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


The number of parameters for radius 10 is 7817172


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.17578071355819702    │
└───────────────────────────┴───────────────────────────┘

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


The number of parameters for radius 15 is 7817172


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.17384971678256989    │
└───────────────────────────┴───────────────────────────┘

TEST Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                          	|  -              	|  252            	

In [ ]:
test_loss_rad_dict = {}

ckpt_dir = "/nfs/turbo/lsa-regier/scratch/roko/output/lightning_logs/checkpoints/MonetDenseXenium"

# Iterate through all files in the directory
for filename in os.listdir(ckpt_dir):
    # Check if the file is a checkpoint file
    if filename.endswith('.ckpt'):
        model_name, hidden_layers, num_kernels, radius, run_name, optim, num_splits = filename.split("__")
        num_splits = num_splits.removesuffix(".ckpt")
        num_splits = num_splits.removeprefix("NUM_SPLITS=")
        if "[512, 512, 512]" not in hidden_layers or "POLAR_AND_DEGREE" != run_name or int(radius) != 30 or int(num_kernels) != 10:
            continue
        with initialize(config_path="../config"):
            # try:
                try:
                    cfg_from_terminal = compose(config_name="configXenium")
                    OmegaConf.update(cfg_from_terminal, "paths.data", "../data")
                    OmegaConf.update(cfg_from_terminal, "paths.root", "/nfs/turbo/lsa-regier/scratch/roko")
                    OmegaConf.update(cfg_from_terminal, "model.kwargs.kernel_size", int(num_kernels))
                    OmegaConf.update(cfg_from_terminal, "model.kwargs.hidden_dimensions", eval(hidden_layers))
                    OmegaConf.update(cfg_from_terminal, "training.logger_name", run_name)
                    OmegaConf.update(cfg_from_terminal, "training.trainer.strategy", "auto")
                    OmegaConf.update(cfg_from_terminal, "radius", int(radius))
                    OmegaConf.update(cfg_from_terminal, "gpus", [0])
                    OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", int(num_splits))
                    print(cfg_from_terminal.training.filepath)
                    # Check if the key already exists in the dictionary
                    if int(radius) in test_loss_rad_dict and int(num_kernels) in test_loss_rad_dict[int(radius)]:
                        print(f"Key ({int(radius)}, {int(num_kernels)}) already exists. Skipping test.")
                        continue
                    output = test(cfg_from_terminal)
                    trainer, l1_losses, inputs_SPATIAL, gene_expressions_SPATIAL, celltypes, test_results_SPATIAL = output
                    # Store the results over (radius, num_kernels) in a nested dict
                    if int(radius) not in test_loss_rad_dict:
                        test_loss_rad_dict[int(radius)] = {}
                    test_loss_rad_dict[int(radius)][int(num_kernels)] = test_results_SPATIAL[0]['test_loss']
                except Exception as e:
                    if "CUDA out of memory" in str(e):
                        OmegaConf.update(cfg_from_terminal, "training.filepath", "${model.name}__${model.kwargs.kernel_size}__${radius}__${training.logger_name}__${optimizer.name}__NUM_SPLITS=" + str(num_splits))
                        OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", 5)
                        print("Increasing splits to 5 due to OOM error.")
                        output = test(cfg_from_terminal)
                        trainer, l1_losses, inputs_SPATIAL, gene_expressions_SPATIAL, celltypes, test_results_SPATIAL = output
                        # Store the results over (radius, num_kernels) in a nested dict
                        if int(radius) not in test_loss_rad_dict:
                            test_loss_rad_dict[int(radius)] = {}
                        test_loss_rad_dict[int(radius)][int(num_kernels)] = test_results_SPATIAL[0]['test_loss']
                    else:
                        raise e


In [ ]:
NON_RESPONSE_FILE = "../spatial/non_response_blank_removed_xenium.txt"
with open(NON_RESPONSE_FILE, "r", encoding="utf-8") as f:
    non_response_genes = f.read().split(',')

non_response_genes = [int(x) for x in non_response_genes]

xenium_df = pd.read_csv("../data/raw/xenium.csv", index_col="cell_id")
 
location_names = ["x_location", "y_location", "z_location", "qv"]

non_response_gene_names = xenium_df.columns[non_response_genes]
# First, combine non_response_gene_names and location_names into a single list
response_gene_names = xenium_df.columns[~xenium_df.columns.isin(non_response_gene_names | location_names)]
combined_names = list(non_response_gene_names) + location_names
# Then, use the ~ operator to get the indexes of columns that are not in the combined list
response_gene_indexes = (~xenium_df.columns.isin(combined_names)).nonzero()[0]


In [4]:
assert ','.join(xenium_df.columns[response_gene_indexes]) == ','.join(response_gene_names), "Response genes don't line up"

In [ ]:
rad = 30
loss_dict = {rad: {}}
for i, gene in enumerate(response_gene_names):
    inputs_SPATIAL_responses = inputs_SPATIAL[:, response_gene_indexes]
    loss_dict[rad][gene] = {"base": torch.mean( (inputs_SPATIAL_responses[:, i] - gene_expressions_BASE[:, i]) ** 2 ).item(), "spatial": torch.mean( (inputs_SPATIAL_responses[:, i] - gene_expressions_SPATIAL[:, i]) ** 2 ).item()}
    loss_dict[rad][gene]["diff"] = loss_dict[rad][gene]["spatial"] - loss_dict[rad][gene]["base"]
    loss_dict[rad][gene]["percent_diff"] = (loss_dict[rad][gene]["diff"]/loss_dict[rad][gene]["base"]) * 100.0
    loss_dict[rad][gene]["sparsity"] = (inputs_SPATIAL_responses[:, i] == 0).float().mean()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

# ICML-style font + layout with bold title and axes
mpl.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "axes.labelsize": 14,
    "axes.titlesize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "axes.linewidth": 1.0,
    "figure.dpi": 300  # High DPI for high-quality output
})

# Data prep
percent_differences = np.array([-loss_dict[30][x]["percent_diff"] for x in loss_dict[30]])

# Define bin edges
bin_width = 1
bins = np.arange(np.floor(percent_differences.min()), np.ceil(percent_differences.max()) + bin_width, bin_width)

# Split bins <0 and >=0
neg_values = percent_differences[percent_differences < 0]
pos_values = percent_differences[percent_differences >= 0]

# Plot
fig, ax = plt.subplots(figsize=(8, 5.2))

# Colorblind-safe colors
color_neg = "#0072B2"  # blue
color_pos = "#E69F00"  # orange

# Histograms
ax.hist(neg_values, bins=bins, color=color_neg, alpha=0.85, label="No Gain")
ax.hist(pos_values, bins=bins, color=color_pos, alpha=0.85, label="Spatial Gain")

# Vertical line at 0%
ax.axvline(0, color='red', linestyle='--', linewidth=1.6, label="No Gain Threshold")

# Labels and legend
ax.set_title("Percent Loss Reduction: Spatial vs Baseline", pad=10, fontweight='bold')  # Bold title
ax.set_xlabel("Percent Reduction in Test Loss", labelpad=8, fontweight='bold')  # Bold x-axis label
ax.set_ylabel("Number of Genes", labelpad=8, fontweight='bold')  # Bold y-axis label
ax.legend(loc="upper right", frameon=False)  # Bold legend

# Grid + layout
ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
fig.tight_layout(pad=1.2)

# Optional save
fig.savefig("xenium_spatial_vs_baseline.png", bbox_inches='tight', dpi=300)  # High DPI for high-quality output

plt.show()


In [ ]:
# Rank genes based on their percent decrease and save in a dictionary
# First, let's create a dictionary with gene names as keys and their percent differences as values
gene_percent_diffs = {x: -loss_dict[30][x]["percent_diff"] for x in loss_dict[30]}

# Now, let's sort the genes based on their percent differences in descending order (largest decrease first)
sorted_genes = sorted(gene_percent_diffs, key=gene_percent_diffs.get, reverse=True)

# Print the top 10 genes with the largest percent decrease
print("Top 10 genes with the largest percent decrease:")
for gene in sorted_genes[:10]:
    print(f"{gene}: {gene_percent_diffs[gene]}% decrease")


In [ ]:
positive_decrease_genes = sum(1 for gene in gene_percent_diffs if gene_percent_diffs[gene] > 0)
total_genes = len(gene_percent_diffs)
percentage_positive_decrease = (positive_decrease_genes / total_genes) * 100

print(f"Percentage of genes with a positive decrease: {percentage_positive_decrease:.2f}%")

In [ ]:
# First, let's sort the genes based on their percent differences in descending order (largest decrease first)
sorted_genes = sorted(loss_dict[30], key=lambda gene: -loss_dict[30][gene]["percent_diff"], reverse=True)

# Now, iterate through each gene in the sorted order
for gene in sorted_genes:
    # Extract percent difference and sparsity for each gene
    percent_diff = loss_dict[30][gene]["percent_diff"]
    sparsity = loss_dict[30][gene]["sparsity"]
    
    # Print the gene name, percent difference, and sparsity in descending order of percent difference
    print(f"Gene Name: {gene}, Percent Difference: {percent_diff}%, Sparsity: {sparsity}")


In [ ]:
loss_dict

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr

# Extract percent differences and sparsity for each gene
percent_diffs = np.array([-loss_dict[30][gene]["percent_diff"] for gene in sorted_genes])
sparsities = np.array([loss_dict[30][gene]["sparsity"] for gene in sorted_genes])

# Compute Pearson correlation
corr, pval = pearsonr(percent_diffs, sparsities)

# Create plot
sns.set(style="whitegrid", font_scale=1.2)
fig, ax = plt.subplots(figsize=(5.5, 3.5))
scatter = ax.scatter(percent_diffs, sparsities, alpha=0.6, edgecolor='k', linewidth=0.3, s=30)

# Axis labels and title
ax.set_title("Predictive Gain for Genes by Sparsity", fontsize=12)
ax.set_xlabel("Percent MSE Reduction from Spatial Information", fontsize=11)
ax.set_ylabel("Gene Sparsity", fontsize=11)

# Add horizontal line at y=0.5
ax.axhline(y=0.5, color='r', linestyle='--')

# Add caption for the horizontal line
ax.text(10, 0.55, ">50% of Expression Reads are 0", fontsize=10, color='r', verticalalignment='top')

# Add correlation in a box
bbox_props = dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white', alpha=0.8)
ax.text(0.95, 0.95,
        f"Pearson r = {corr:.2f}\np = {pval:.1e}",
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment='top',
        horizontalalignment='right',
        bbox=bbox_props)

plt.tight_layout()
plt.savefig('sparsity_scatter_Xenium.png')
plt.show()


In [ ]:
test_loss_rad_dict = {}
loss_dict = {}

ckpt_dir = "/nfs/turbo/lsa-regier/scratch/roko/output/lightning_logs/checkpoints/MonetDenseXenium"

# Iterate through all files in the directory
for filename in os.listdir(ckpt_dir):
    # Check if the file is a checkpoint file
    if filename.endswith('.ckpt'):
        model_name, hidden_layers, num_kernels, radius, run_name, optim, num_splits = filename.split("__")
        num_splits = num_splits.removesuffix(".ckpt")
        num_splits = num_splits.removeprefix("NUM_SPLITS=")
        if "[512, 512, 512]" not in hidden_layers or "POLAR_AND" not in run_name or int(num_kernels) != 10:
            continue
        with initialize(config_path="../config"):
            # try:
                try:
                    cfg_from_terminal = compose(config_name="configXenium")
                    OmegaConf.update(cfg_from_terminal, "paths.data", "../data")
                    OmegaConf.update(cfg_from_terminal, "paths.root", "/nfs/turbo/lsa-regier/scratch/roko")
                    OmegaConf.update(cfg_from_terminal, "model.kwargs.kernel_size", int(num_kernels))
                    OmegaConf.update(cfg_from_terminal, "model.kwargs.hidden_dimensions", eval(hidden_layers))
                    OmegaConf.update(cfg_from_terminal, "training.logger_name", run_name)
                    OmegaConf.update(cfg_from_terminal, "training.trainer.strategy", "auto")
                    OmegaConf.update(cfg_from_terminal, "radius", int(radius))
                    OmegaConf.update(cfg_from_terminal, "gpus", [1])
                    OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", int(num_splits))
                    print(cfg_from_terminal.training.filepath)
                    # Check if the key already exists in the dictionary
                    if int(radius) in test_loss_rad_dict:
                        print(f"Key ({int(radius)}, {int(num_kernels)}) already exists. Skipping test.")
                        continue
                    output = test(cfg_from_terminal)
                    trainer, l1_losses, inputs_SPATIAL, gene_expressions_SPATIAL, celltypes, test_results_SPATIAL = output
                    # Store the results over (radius, num_kernels) in a nested dict
                    if int(radius) not in test_loss_rad_dict:
                        test_loss_rad_dict[int(radius)] = {}
                    test_loss_rad_dict[int(radius)] = test_results_SPATIAL[0]['test_loss']
                except Exception as e:
                    if "CUDA out of memory" in str(e):
                        OmegaConf.update(cfg_from_terminal, "training.filepath", "${model.name}__${model.kwargs.kernel_size}__${radius}__${training.logger_name}__${optimizer.name}__NUM_SPLITS=" + str(num_splits))
                        OmegaConf.update(cfg_from_terminal, "datasets.dataset.splits", 5)
                        print("Increasing splits to 5 due to OOM error.")
                        output = test(cfg_from_terminal)
                        trainer, l1_losses, inputs_SPATIAL, gene_expressions_SPATIAL, celltypes, test_results_SPATIAL = output
                        # Store the results over (radius, num_kernels) in a nested dict
                        if int(radius) not in test_loss_rad_dict:
                            test_loss_rad_dict[int(radius)] = {}
                        test_loss_rad_dict[int(radius)] = test_results_SPATIAL[0]['test_loss']
                    else:
                        raise e
    loss_dict[radius] = {}
    for i, gene in enumerate(response_gene_names):
        inputs_SPATIAL_responses = inputs_SPATIAL[:, response_gene_indexes]
        loss_dict[radius][gene] = {"base": torch.mean( (inputs_SPATIAL_responses[:, i] - gene_expressions_BASE[:, i]) ** 2 ).item(), "spatial": torch.mean( (inputs_SPATIAL_responses[:, i] - gene_expressions_SPATIAL[:, i]) ** 2 ).item()}
        loss_dict[radius][gene]["diff"] = loss_dict[radius][gene]["spatial"] - loss_dict[radius][gene]["base"]
        loss_dict[radius][gene]["percent_diff"] = (loss_dict[radius][gene]["diff"]/loss_dict[radius][gene]["base"]) * 100.0


In [ ]:
loss_dict

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json


plt.style.use('default')

# Assuming these variables are defined elsewhere in your code
# radius_values, genes, data, response_indexes

fig, axes = plt.subplots(2, 3, figsize=(24, 28))

quarter = 0

# Initialize an empty array to store the full loss data
full_loss_array = np.array([])

for ax in axes.ravel():
    
    quarter += 1
    radius_values = list(range(0, 31, 5))
    n = len(response_gene_indexes)
    genes = response_gene_names[n * (quarter - 1) // 6 : n * quarter // 6]
    
    # Initialize an empty array to store the loss data for the current quarter
    loss_array = np.array([])
    
    # Loop through each gene and radius to build the loss array
    for gene in genes:
        for rad in radius_values:
            # Extract the percent difference for the current gene and radius
            percent_diff = loss_dict[str(rad)][gene]["percent_diff"]
            # Append the negative of the percent difference to the loss array
            loss_array = np.append(loss_array, -percent_diff)
    
    # Reshape the loss array to match the dimensions of the genes and radius values
    loss_array = loss_array.reshape(len(genes), len(radius_values))
    
    # If this is the first quarter, initialize the full_loss_array
    if full_loss_array.size == 0:
        full_loss_array = loss_array.T
    else:
        # Append the current loss array to the full loss array
        full_loss_array = np.append(full_loss_array, loss_array.T, axis=1)
    
    # Plotting the loss array
    im = ax.imshow(loss_array, cmap='seismic', 
                   vmin=-np.max(np.abs(full_loss_array)), 
                   vmax=np.max(np.abs(full_loss_array)))
    
    # Adjusting the aspect ratio to make cells wider horizontally
    ax.set_aspect(0.8)
    
    # Setting x-axis ticks and labels with a label size of 10
    ax.set_xticks(np.arange(len(radius_values)))
    ax.set_xticklabels(list(radius_values), fontsize=14)
    
    # Setting y-axis ticks and labels with a custom font dict
    gene_font = {'fontsize': 14, 'fontweight': 'bold', 'color': 'navy'}  # Increased font size for gene names
    ax.set_yticks(np.arange(len(genes)))
    ax.set_yticklabels(genes, fontdict=gene_font)
    
    # Adjusting tick font sizes for both axes
    ax.tick_params(axis='both', which='major')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    # Dynamically adjusting text color and font weight based on cell value
    for i in range(len(genes)):
        for j in range(len(radius_values)):
            text_color = 'red' if loss_array[i, j] < 0 else 'black'
            # Making text bold if the loss is greater than 10 (adjust the condition as needed)
            fontweight = 'bold' if loss_array[i, j] > 10 else 'normal'
            ax.text(j, i, f"{loss_array[i, j]:.2f}",
                    ha="center", va="center", color=text_color,
                    fontsize=15, fontweight=fontweight)

fig.tight_layout(w_pad=0.5)  # Adjusting the width padding to move columns closer together
plt.savefig("spatial_vertical_full_Xenium.png", dpi=200)
plt.show()